# Datenaufbereitung

In diesem Notebook stellen wir den aufgeräumten Datensatz her, der den Karten und Modellen im Notebook über Autos und Landschaft zugrundeliegt. Die hier angewandten Techniken beinhalten:

- Download und Auslesen von Excel-Tabellen
- Zusammenfügen von Daten aus unterschiedlichen Quellen
- Umgang mit georeferenzierten Raster, und Vektordaten
- weil's geht: Webcrawling von tabellarischen Daten aus der Wikipedia

In [ ]:
import urllib.request
import os
import zipfile
import pandas as pd
import geopandas as gpd
import plotly.express as px
import rioxarray
from pyproj import CRS
import rpy2
%load_ext rpy2.ipython

In [ ]:
%%R

suppressMessages({
    library(tidyverse)
    library(readxl)
    library(terra)
    library(spatialEco)
    library(raster)
})

# Download der öffentlichen Daten

- **Zulassungsdaten:**  
  Kraftfahrtbundesamt  
  https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ1/fz1_2022.xlsx

- **Landkreisgrenzen und Höhenmodell:**  
  Geodatenzentrum des Bundesamtes für Kartographie und Geodäsie  
  https://daten.gdz.bkg.bund.de/produkte/vg/vg2500/aktuell/vg2500_12-31.utm32s.shape.zip  
  https://daten.gdz.bkg.bund.de/produkte/dgm/dgm200/aktuell/dgm200.utm32s.geotiff.zip

- **Haushaltseinkommen pro Kopf:**  
  Statistische Ämter des Bundes und der Länder  
  https://www.statistikportal.de/sites/default/files/2022-10/vgrdl_r2b3_bs2021_0.xlsx
  

In [ ]:
# file download:
try:
    os.mkdir("download")
    os.mkdir("data")
except FileExistsError as error:
    print("Data dirs already exist.")

urllib.request.urlretrieve("https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ1/fz1_2022.xlsx?__blob=publicationFile&v=5", "download/Zulassungen.xlsx")
urllib.request.urlretrieve("https://daten.gdz.bkg.bund.de/produkte/vg/vg2500/aktuell/vg2500_12-31.utm32s.shape.zip", "download/Kreisgrenzen.zip")
urllib.request.urlretrieve("https://daten.gdz.bkg.bund.de/produkte/dgm/dgm200/aktuell/dgm200.utm32s.geotiff.zip", "download/Höhenmodell.zip")
urllib.request.urlretrieve("https://www.statistikportal.de/sites/default/files/2022-10/vgrdl_r2b3_bs2021_0.xlsx", "download/Einkommen.xlsx")

with zipfile.ZipFile("download/Kreisgrenzen.zip", "r") as zip_ref:
    zip_ref.extractall("download/Kreisgrenzen")
    
with zipfile.ZipFile("download/Höhenmodell.zip", "r") as zip_ref:
    zip_ref.extractall("download/Höhenmodell")

# Zulassungsdaten

Gesucht ist je Kreis die Anzahl der PKW-Zulassungen, unterschieden nach Hubraum und nach Euro-Abgasnorm.
Besonders der Hubraum dient uns als Anhaltspunkt, da wir davon ausgehen, dass abgesehen von der Hügeligkeit der umgebenden Landschaft keine anderen, von Kreis zu Kreis systematisch variierenden Faktoren existieren, die vorhersagen würden, dass hier stärkere PKWs als dort notwendig wären.

In [ ]:
%%R
pkw_cm3 <- read_excel("download/Zulassungen.xlsx",
                      sheet = "FZ1.1",
                      skip = 8,
                      .name_repair = ~ vctrs::vec_as_names(..., repair = "unique", quiet = TRUE))

colnames(pkw_cm3) <- str_replace_all(colnames(pkw_cm3), "[^a-zA-Z0-9]", "_")

pkw_cm3 <- pkw_cm3 |> 
  transmute( id_bezirk = Statistische_Kennziffer_und_Zulassungsbezirk,
             krs_code = str_extract(id_bezirk, "^[0-9]+"),
             id_bezirk = NULL,
             pkw_gesamt = insgesamt___9,
             max_cm3_1399 = Hubraum__bis___1_399_cm_,
             max_cm3_1999 = `1_400__bis__1_999_cm_`,
             cm3_over_1999 = `2_000__und__mehr_cm_`,
             allrad = und_zwar___mit__Allrad___antrieb ) |> 
  filter(!is.na(krs_code))



pkw_euro <- read_excel("download/Zulassungen.xlsx",
                       sheet = "FZ1.2",
                       skip = 8,
                       .name_repair = ~ vctrs::vec_as_names(..., repair = "unique", quiet = TRUE))

colnames(pkw_euro) <- str_replace_all(colnames(pkw_euro), "[^a-zA-Z0-9]", "_")

pkw_euro <- pkw_euro |> 
  transmute( krs_code = str_extract(`___3`, "^[0-9]+"),
             euro_1 = Euro_1___12,
             euro_2 = Euro_2___13,
             euro_3 = Euro_3___14,
             euro_4 = Euro_4___15,
             euro_5 = Euro_5___16,
             euro_6 = Euro_6___17 ) |> 
  filter(!is.na(krs_code))

Wir erstellen den Gesamtdatensatz `data` und fügen als Erstes die Zulassungsdaten hinzu:

In [ ]:
%%R

data <- pkw_cm3 |> 
  inner_join(pkw_euro, by = c("krs_code")) |> 

  # zusätzlich zu Absolutzahlen die Anteile am Gesamtbestand:
  mutate( 
#      max_cm3_1399_pct = max_cm3_1399 / pkw_gesamt * 100,
#      max_cm3_1999_pct = max_cm3_1999 / pkw_gesamt * 100,
      cm3_over_1999_pct = cm3_over_1999 / pkw_gesamt * 100,
#      euro_1_pct = euro_1 / pkw_gesamt * 100,
#      euro_2_pct = euro_2 / pkw_gesamt * 100,
#      euro_3_pct = euro_3 / pkw_gesamt * 100,
#      euro_4_pct = euro_4 / pkw_gesamt * 100,
#      euro_5_pct = euro_5 / pkw_gesamt * 100,
      euro_6_pct = euro_6 / pkw_gesamt * 100,
      allrad_pct = allrad / pkw_gesamt * 100
      )
print(data)

# Haushaltseinkommen

Für jeden Kreis bestehen Daten über das durchschnittliche verfügbare Haushaltseinkommen pro Kopf. Wir ordnen diesen Wert den Kreisen in unserem Datensatz zu:

In [ ]:
%%R
income <- read_excel("download/Einkommen.xlsx",
                     sheet = "2.4",
                     range = "A5:AH451" ) |>
  slice(-1) |> 
  transmute( krs_code = `Regional-schlüssel`,
             nuts3 = `NUTS 3`,
             income = `2020`) |> 
  # Länder & Reg.-Bez. ignorieren:
  filter(nuts3 == 3) |> 
  # die 2-stelligen Codes von HH und B mit Nullen auffüllen
  # (mit Dank an dieses krumme Code-Format, das Nullen an Anfang und Ende erlaubt
  # und dann auch noch inkonsistent benutzt wird)
  mutate(krs_code = krs_code |> map_chr(~ paste0(., paste0(rep("0", 5-nchar(.)), collapse = "")))) |> 
  dplyr::select(-nuts3)

print(head(income))
    
data <- data |> 
  inner_join( income, by = "krs_code" )

column_names <- colnames(data)

# Umrisse der (Land-)Kreise

Um zu erfahren, wie rau das Gelände durchschnittlich in jedem Kreis ist, brauchen wir seinen Umriss, um damit den betreffenden Teil aus dem deutschlandweiten Höhenraster "auszustechen" und in ihm den Durchschnittswert zu berechnen. Außerdem benutzen wir diese Umrissdaten gleich für Kartendarstellungen mit.

In [ ]:
kreise = gpd.read_file("download/Kreisgrenzen/vg2500_12-31.utm32s.shape/vg2500/VG2500_KRS.shp").\
           loc[:,["AGS", "GEN", "geometry"]].\
           rename({"AGS": "krs_code",
                   "GEN": "name"}, axis = 1)
kreise.set_index("krs_code", inplace = True)
kreise = kreise.to_crs(epsg = "4326")

In [ ]:
kreise.head()

In [ ]:
import rpy2.robjects as robjects

# Datensatz aus R überführen, Metadaten und Datentypen korrigieren:
data = pd.DataFrame(robjects.globalenv["data"]).transpose()
data.columns = robjects.globalenv["column_names"]
data.set_index("krs_code", inplace = True)
data = data.astype(dict(pkw_gesamt=int, max_cm3_1399=int, max_cm3_1999=int, cm3_over_1999=int,
                        euro_1=int, euro_2=int, euro_3=int, euro_4=int, euro_5=int, euro_6=int,
                        cm3_over_1999_pct=float, euro_6_pct=float, allrad_pct=float,
                        allrad=int, income=int) )

# mit Geodaten zusammenführen:
data = pd.merge(left  = data,   left_on  = "krs_code",
                right = kreise, right_on = "krs_code")
data = gpd.GeoDataFrame(data)

data.head()

# Höhenmodell für die Fläche Deutschlands

Mit dem *Terrain Ruggedness Index (TRI)* ([Riley et al., 1999](#Bibliografie)) existiert ein Maß für die Rauhheit oder Hügeligkeit einer Landschaft. Es basiert auf einem Höhenmodell, also einem Raster, das für jeden Punkt in einer gewissen Auflösung die Höhe ü. M. angibt. Wir laden das Höhenmodell in R und erstellen daraus ein Raster, das den TRI repräsentiert:

In [ ]:
%%R
dem_d <- terra::rast("download/Höhenmodell/dgm200.utm32s.geotiff/dgm200/dgm200_utm32s.tif")
ruggedness_d <- spatialEco::tri(dem_d) # dieser Prozess dauert wenige Minuten
terra::writeRaster(ruggedness_d, "data/TRI_DEM200_Deutschland.tif", overwrite = TRUE)

In Python laden wir das Raster und berechnen je Kreis einen Durchschnittswert:

In [ ]:
ruggedness_d = rioxarray.open_rasterio("data/TRI_DEM200_Deutschland.tif")
ruggedness_d = ruggedness_d.rio.set_crs(25832)
ruggedness_d = ruggedness_d.rio.reproject("EPSG:4326")

rug_averages = []

for i in range(len(data)):
  # nimm jeden Kreis-Umriss,
  this_geo = [data.iloc[i]["geometry"]]
  # isoliere den Teil des TRI-Rasters, der darunter liegt
  this_raster = ruggedness_d.rio.clip(this_geo)
  # und ermittle durchschnittliche Rauhheit
  this_avg = float(this_raster.mean())
  rug_averages.append(this_avg)

data["m_rugged"] = rug_averages

data.head()

In [ ]:
data = data.drop("geometry", axis = 1)

import csv
data.to_csv("data/krs_pkw_rug.csv",
            index = True,
            quoting = csv.QUOTE_NONNUMERIC)

# Bibliografie

Zum Nachschlagen der Literaturverweise auf wissenschaftliche Arbeiten:

Riley, S. J., DeGloria, S. D., & Elliot, R. (1999). [Index that quantifies topographic heterogeneity.](http://download.osgeo.org/qgis/doc/reference-docs/Terrain_Ruggedness_Index.pdf) *Intermountain Journal of Sciences*, 5(1-4), 23-27.